# Kaggle Competition
# House Prices: Advanced Regression Techniques

### Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

### Metric
Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. (Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.)

### Importing necessary libraries

In [4]:
# import neccessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# configurations
%matplotlib.inline
sns.set_style('whitegrid')

UsageError: Line magic function `%matplotlib.inline` not found.


In [5]:
# check versions of the imported libraries
np.__version__,pd.__version__,sns.__version__

('1.17.0', '0.25.0', '0.9.0')

### Configurations

In [6]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')
# display all rows
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)

### Importing Dataset

In [7]:
# importing train and test dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [8]:
# print dimensions of the train dataset
print('# rows in dataset    : ',train.shape[0])
print('# columns in dataset : ',train.shape[1])

# rows in dataset    :  1460
# columns in dataset :  81


In [9]:
# print dimensions of the dataset
print('# rows in dataset    : ',test.shape[0])
print('# columns in dataset : ',test.shape[1])

# rows in dataset    :  1459
# columns in dataset :  80


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

`train` dataset has 81 features with some datapoints missing

In [11]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


### Data pre-processing

In [12]:
# remove duplicates
train.drop_duplicates(inplace = True)

In [13]:
# print dimensions of the train dataset
print('# rows in dataset    : ',train.shape[0])
print('# columns in dataset : ',train.shape[1])

# rows in dataset    :  1460
# columns in dataset :  81


Thus, there are no duplicate rows in the train dataset

In [14]:
# check for missing values
train.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [15]:
# calculate %age of missing values in each feature
null_features = round((train.isnull().sum() / train.shape[0])*100, 2)
null_features.sort_values(ascending=False)

PoolQC           99.52
MiscFeature      96.30
Alley            93.77
Fence            80.75
FireplaceQu      47.26
LotFrontage      17.74
GarageCond        5.55
GarageType        5.55
GarageYrBlt       5.55
GarageFinish      5.55
GarageQual        5.55
BsmtExposure      2.60
BsmtFinType2      2.60
BsmtFinType1      2.53
BsmtCond          2.53
BsmtQual          2.53
MasVnrArea        0.55
MasVnrType        0.55
Electrical        0.07
Utilities         0.00
YearRemodAdd      0.00
MSSubClass        0.00
Foundation        0.00
ExterCond         0.00
ExterQual         0.00
Exterior2nd       0.00
Exterior1st       0.00
RoofMatl          0.00
RoofStyle         0.00
YearBuilt         0.00
LotConfig         0.00
OverallCond       0.00
OverallQual       0.00
HouseStyle        0.00
BldgType          0.00
Condition2        0.00
BsmtFinSF1        0.00
MSZoning          0.00
LotArea           0.00
Street            0.00
Condition1        0.00
Neighborhood      0.00
LotShape          0.00
LandContour

Let's remove all features that have more than 80% missing values.
<br>We will remove feature 'Id' as well since it is a unique identifier

In [16]:
cols_to_remove = ['PoolQC','MiscFeature','Alley','Fence','Id'] # list of columns to remove
train.drop(columns=cols_to_remove, inplace=True) # drop the columns

No we will analyse each feature with missing value one by one

In [17]:
# analysing 'FireplaceQu'
print('missing values : ',train['FireplaceQu'].isnull().sum())

# replacing missing values with mode
train['FireplaceQu'] = train['FireplaceQu'].fillna(train['FireplaceQu'].mode()[0])

missing values :  690


In [18]:
# analysing 'LotFrontage'
# since 'LotFrontage' is a float datatype, we will fill the missing values with mean
train['LotFrontage'] = train['LotFrontage'].fillna(train['LotFrontage'].mean())

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 76 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1452 no

In [20]:
# analysing 'GarageType', 'GarageFinish', 'GarageCond', 'GarageQual', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType', 'Electrical'
# the above features are object datatype
# we will fill the missing values with mode
features_to_fill = ['GarageType', 'GarageFinish', 'GarageCond', 'GarageQual', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType', 'Electrical']
# replacing missing values
for i in features_to_fill:
    train[i] = train[i].fillna(train[i].mode()[0])



In [21]:
# analysing 'GarageYrBlt' & 'MasVnrArea'
# the above features are float type
# we will replace the missing values with mean
train['GarageYrBlt'] = train['GarageYrBlt'].fillna(train['GarageYrBlt'].mode()[0])
train['MasVnrArea'] = train['MasVnrArea'].fillna(train['MasVnrArea'].mode()[0])

In [22]:
# calculate %age of missing values in each feature
round((train.isnull().sum() / train.shape[0])*100, 2).sort_values(ascending=False)
# null_features.sort_values(ascending=False)

SalePrice        0.0
Foundation       0.0
RoofMatl         0.0
Exterior1st      0.0
Exterior2nd      0.0
MasVnrType       0.0
MasVnrArea       0.0
ExterQual        0.0
ExterCond        0.0
BsmtQual         0.0
SaleCondition    0.0
BsmtCond         0.0
BsmtExposure     0.0
BsmtFinType1     0.0
BsmtFinSF1       0.0
BsmtFinType2     0.0
BsmtFinSF2       0.0
BsmtUnfSF        0.0
RoofStyle        0.0
YearRemodAdd     0.0
YearBuilt        0.0
OverallCond      0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
Street           0.0
LotShape         0.0
LandContour      0.0
Utilities        0.0
LotConfig        0.0
LandSlope        0.0
Neighborhood     0.0
Condition1       0.0
Condition2       0.0
BldgType         0.0
HouseStyle       0.0
OverallQual      0.0
TotalBsmtSF      0.0
Heating          0.0
HeatingQC        0.0
GarageYrBlt      0.0
GarageCars       0.0
GarageArea       0.0
GarageQual       0.0
GarageCond       0.0
PavedDrive       0.0
WoodDeckSF       0.0
OpenPorchSF  

Therefore, all features with missing values have been treated

In [23]:
# print dimensions of the train dataset
print('# rows in dataset    : ',train.shape[0])
print('# columns in dataset : ',train.shape[1])

# rows in dataset    :  1460
# columns in dataset :  76


## Processing the test dataset

In [24]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            107 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-

In [25]:
# print dimensions of the test dataset
print('# rows in dataset    : ',test.shape[0])
print('# columns in dataset : ',test.shape[1])

# rows in dataset    :  1459
# columns in dataset :  80


In [26]:
# calculate the percentage of missing values in all features
round((test.isnull().sum()/test.shape[0])*100, 2).sort_values(ascending=False)

PoolQC           99.79
MiscFeature      96.50
Alley            92.67
Fence            80.12
FireplaceQu      50.03
LotFrontage      15.56
GarageCond        5.35
GarageQual        5.35
GarageYrBlt       5.35
GarageFinish      5.35
GarageType        5.21
BsmtCond          3.08
BsmtQual          3.02
BsmtExposure      3.02
BsmtFinType1      2.88
BsmtFinType2      2.88
MasVnrType        1.10
MasVnrArea        1.03
MSZoning          0.27
BsmtHalfBath      0.14
Utilities         0.14
Functional        0.14
BsmtFullBath      0.14
BsmtFinSF2        0.07
BsmtFinSF1        0.07
Exterior2nd       0.07
BsmtUnfSF         0.07
TotalBsmtSF       0.07
SaleType          0.07
Exterior1st       0.07
KitchenQual       0.07
GarageArea        0.07
GarageCars        0.07
HouseStyle        0.00
LandSlope         0.00
MSSubClass        0.00
LotArea           0.00
Street            0.00
LotShape          0.00
LandContour       0.00
LotConfig         0.00
Neighborhood      0.00
BldgType          0.00
Condition1 

Let's remove all features that have more than 80% missing values

In [27]:
cols_to_remove = ['PoolQC','MiscFeature','Alley','Fence','Id'] # list of columns to remove
test.drop(columns=cols_to_remove, inplace=True) # drop the columnsLet's remove all features that have more than 80% missing values

In [28]:
# treating all the missing data in test dataset
# create a list of all features with missing values
cols_to_fill = ['FireplaceQu','LotFrontage','GarageCond','GarageQual','GarageYrBlt','GarageFinish',
'GarageType','BsmtCond','BsmtQual','BsmtExposure','BsmtFinType1','BsmtFinType2','MasVnrType',
'MasVnrArea','MSZoning','BsmtHalfBath','Utilities','Functional','BsmtFullBath','BsmtUnfSF',
'SaleType','BsmtFinSF2','BsmtFinSF1','Exterior2nd','Exterior1st','TotalBsmtSF','GarageCars',
'KitchenQual','GarageArea']
test_objects = [] # list to capture all object dtypes
test_num = [] # list to capture all float64 dtypes

In [29]:
# function for segmentation & treatment of missing values in features by dtype
for i in cols_to_fill: 
    if test[i].dtype == 'object':
        test[i] = test[i].fillna(test[i].mode()[0])
    elif test[i].dtype == 'float64':
        test[i] = test[i].fillna(test[i].mean())
    else:
        print('feature {} of dtype {} is neither object nor float'.format(i, test[i].dtype))

In [30]:
# calculate the percentage of missing values in all features
round((test.isnull().sum()/test.shape[0])*100, 2).sort_values(ascending=False)

SaleCondition    0.0
YearRemodAdd     0.0
RoofMatl         0.0
Exterior1st      0.0
Exterior2nd      0.0
MasVnrType       0.0
MasVnrArea       0.0
ExterQual        0.0
ExterCond        0.0
Foundation       0.0
BsmtQual         0.0
BsmtCond         0.0
BsmtExposure     0.0
BsmtFinType1     0.0
BsmtFinSF1       0.0
BsmtFinType2     0.0
BsmtFinSF2       0.0
RoofStyle        0.0
YearBuilt        0.0
TotalBsmtSF      0.0
OverallCond      0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
Street           0.0
LotShape         0.0
LandContour      0.0
Utilities        0.0
LotConfig        0.0
LandSlope        0.0
Neighborhood     0.0
Condition1       0.0
Condition2       0.0
BldgType         0.0
HouseStyle       0.0
OverallQual      0.0
BsmtUnfSF        0.0
Heating          0.0
SaleType         0.0
GarageType       0.0
GarageFinish     0.0
GarageCars       0.0
GarageArea       0.0
GarageQual       0.0
GarageCond       0.0
PavedDrive       0.0
WoodDeckSF       0.0
OpenPorchSF  

## Now concatenate both the train & test dataset
Since the categorical features in both datasets have different categories in them, therefore creating dummies individually would not fair well for us.
<br>Therefore, we will concatenate both train and test datasets, create dummies, then separate them again.

In [31]:
# print dimensions of the train dataset
print('# rows in dataset    : ',train.shape[0])
print('# columns in dataset : ',train.shape[1])

# rows in dataset    :  1460
# columns in dataset :  76


In [32]:
# print dimensions of the test dataset
print('# rows in dataset    : ',test.shape[0])
print('# columns in dataset : ',test.shape[1])

# rows in dataset    :  1459
# columns in dataset :  75


In [33]:
df = pd.concat([train,test], axis=0)
df.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,RoofMatl,RoofStyle,SaleCondition,SalePrice,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,1Fam,TA,No,706.0,0.0,GLQ,Unf,1.0,0.0,Gd,150.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,Gd,0,PConc,2,Typ,548.0,2.0,TA,RFn,TA,Attchd,2003.0,1710,1,GasA,Ex,2Story,1,Gd,Lvl,Gtl,8450,Inside,65.0,Reg,0,60,RL,196.0,BrkFace,0,2,CollgCr,61,5,7,Y,0,CompShg,Gable,Normal,208500.0,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,3,1Fam,TA,Gd,978.0,0.0,ALQ,Unf,0.0,1.0,Gd,284.0,Y,Feedr,Norm,SBrkr,0,TA,TA,MetalSd,MetalSd,TA,1,CBlock,2,Typ,460.0,2.0,TA,RFn,TA,Attchd,1976.0,1262,0,GasA,Ex,1Story,1,TA,Lvl,Gtl,9600,FR2,80.0,Reg,0,20,RL,0.0,None,0,5,Veenker,0,8,6,Y,0,CompShg,Gable,Normal,181500.0,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007
2,920,866,0,3,1Fam,TA,Mn,486.0,0.0,GLQ,Unf,1.0,0.0,Gd,434.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,TA,1,PConc,2,Typ,608.0,2.0,TA,RFn,TA,Attchd,2001.0,1786,1,GasA,Ex,2Story,1,Gd,Lvl,Gtl,11250,Inside,68.0,IR1,0,60,RL,162.0,BrkFace,0,9,CollgCr,42,5,7,Y,0,CompShg,Gable,Normal,223500.0,WD,0,Pave,6,920.0,AllPub,0,2001,2002,2008
3,961,756,0,3,1Fam,Gd,No,216.0,0.0,ALQ,Unf,1.0,0.0,TA,540.0,Y,Norm,Norm,SBrkr,272,TA,TA,Wd Sdng,Wd Shng,Gd,1,BrkTil,1,Typ,642.0,3.0,TA,Unf,TA,Detchd,1998.0,1717,0,GasA,Gd,2Story,1,Gd,Lvl,Gtl,9550,Corner,60.0,IR1,0,70,RL,0.0,None,0,2,Crawfor,35,5,7,Y,0,CompShg,Gable,Abnorml,140000.0,WD,0,Pave,7,756.0,AllPub,0,1915,1970,2006
4,1145,1053,0,4,1Fam,TA,Av,655.0,0.0,GLQ,Unf,1.0,0.0,Gd,490.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,TA,1,PConc,2,Typ,836.0,3.0,TA,RFn,TA,Attchd,2000.0,2198,1,GasA,Ex,2Story,1,Gd,Lvl,Gtl,14260,FR2,84.0,IR1,0,60,RL,350.0,BrkFace,0,12,NoRidge,84,5,8,Y,0,CompShg,Gable,Normal,250000.0,WD,0,Pave,9,1145.0,AllPub,192,2000,2000,2008


In [34]:
# print dimensions of the df dataset
print('# rows in dataset    : ',df.shape[0])
print('# columns in dataset : ',df.shape[1])

# rows in dataset    :  2919
# columns in dataset :  76


In [35]:
# calculate the percentage of missing values in all features
round((df.isnull().sum()/df.shape[0])*100, 2).sort_values(ascending=False)

SalePrice        49.98
YrSold            0.00
ExterCond         0.00
ExterQual         0.00
Exterior1st       0.00
Exterior2nd       0.00
FireplaceQu       0.00
Fireplaces        0.00
Foundation        0.00
FullBath          0.00
Electrical        0.00
Functional        0.00
GarageArea        0.00
GarageCars        0.00
GarageCond        0.00
GarageFinish      0.00
GarageQual        0.00
GarageType        0.00
EnclosedPorch     0.00
Condition2        0.00
GrLivArea         0.00
Condition1        0.00
2ndFlrSF          0.00
3SsnPorch         0.00
BedroomAbvGr      0.00
BldgType          0.00
BsmtCond          0.00
BsmtExposure      0.00
BsmtFinSF1        0.00
BsmtFinSF2        0.00
BsmtFinType1      0.00
BsmtFinType2      0.00
BsmtFullBath      0.00
BsmtHalfBath      0.00
BsmtQual          0.00
BsmtUnfSF         0.00
CentralAir        0.00
GarageYrBlt       0.00
HalfBath          0.00
YearRemodAdd      0.00
Neighborhood      0.00
OverallCond       0.00
OverallQual       0.00
PavedDrive 

In [36]:
df['SalePrice'].dtype

dtype('float64')

Since 'SalePrice' is our dependent variable and also a float dtype, we will not treat it

In [37]:
# function for segmentation of features by dtype
df_objects = []
df_floats = []
df_ints = []

for i in df.columns: 
    if df[i].dtype == 'object':
        df_objects.append(i)
#         df[i] = df[i].fillna(df[i].mode()[0])
    elif df[i].dtype == 'float64':
        df_floats.append(i)
#         df[i] = df[i].fillna(df[i].mean())
    elif df[i].dtype == 'int64':
        df_ints.append(i)
#         df[i] = df[i].fillna(df[i].mean())
    else:
        print('feature {} of dtype {} is neither object nor float'.format(i, df[i].dtype))

In [38]:
final_df = pd.get_dummies(df, columns=df_objects, drop_first=True)

In [39]:
# print dimensions of the final_df dataset
print('# rows in dataset    : ',final_df.shape[0])
print('# columns in dataset : ',final_df.shape[1])

# rows in dataset    :  2919
# columns in dataset :  237


In [40]:
final_df.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GarageYrBlt,GrLivArea,HalfBath,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MSSubClass,MasVnrArea,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PoolArea,SalePrice,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,CentralAir_Y,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LotShape_IR2,LotShape_IR3,LotShape_Reg,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,PavedDrive_P,PavedDrive_Y,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Pave,Utilities_NoSeWa
0,856,854,0,3,706.0,0.0,1.0,0.0,150.0,0,0,2,548.0,2.0,2003.0,1710,1,1,8450,65.0,0,60,196.0,0,2,61,5,7,0,208500.0,0,8,856.0,0,2003,2003,2008,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0

In [41]:
# remove duplicate rows if any
final_df =final_df.loc[:,~final_df.columns.duplicated()]

In [42]:
# print dimensions of the final_df dataset
print('# rows in dataset    : ',final_df.shape[0])
print('# columns in dataset : ',final_df.shape[1])

# rows in dataset    :  2919
# columns in dataset :  237


In [43]:
# separate test and train datasets
df_train=final_df.iloc[:1460,:] # train dataset
df_test=final_df.iloc[1460:,:] # train dataset

In [44]:
# drop 'SalePrice' from test dataset
df_test.drop(columns = ['SalePrice'], inplace=True)

In [45]:
# print dimensions of the train dataset
print('# rows in df_train dataset    : ',df_train.shape[0])
print('# columns in df_train dataset : ',df_train.shape[1])

# print dimensions of the test dataset
print('# rows in df_test dataset     : ',df_test.shape[0])
print('# columns in df_test dataset  : ',df_test.shape[1])

# rows in df_train dataset    :  1460
# columns in df_train dataset :  237
# rows in df_test dataset     :  1459
# columns in df_test dataset  :  236


Therefore, we have succesfully segmented train and test datasets with the dummy variables

## Modelling

In [46]:
X_train = df_train.drop('SalePrice', axis=1)
y_train = df_train["SalePrice"]

In [48]:
import xgboost
xgb_classifier = xgboost.XGBRegressor()
xgb_classifier.fit(X_train, y_train)

[00:11:49] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [50]:
# save model as pkl file
import pickle
filename = 'final_model.pkl'
pickle.dump(xgb_classifier, open(filename, 'wb'))

In [51]:
y_pred = xgb_classifier.predict(df_test)
y_pred

array([120705.23, 160660.23, 176833.4 , ..., 172080.42, 123883.91,
       239341.78], dtype=float32)

In [52]:
# create sample submission file
pred = pd.DataFrame(y_pred)
submission_df = pd.read_csv('sample_submission.csv')

datasets = pd.concat([submission_df['Id'],pred], axis=1)
datasets.columns = ['Id','SalePrice']
datasets.to_csv('sample_submission.csv',index=False)